# Pre-Processing of data for Team 8


Dongzhe Zhang, Fernanda Lin, Hongyang Liu, Lyufan Pan and Mansi Tolia

This notebook is to be used before the notebook **"BA770_B_Team_8_Final_Notebook"** which processes data from Yelp.

This shows the pre-processing steps needed to achieve the final results.

Creating Chosen_cities table by calculating calculating average weekly restaurant check-ins per city

Organized cities by average check-ins in descending order. 

Selected the cities with the highest average check-ins and limited to 5

In [1]:
%%bq query

SELECT city, 
      COUNT(DISTINCT busniness.business_id) AS total_number_of_restaurants, 
      SUM(checkins.checkins) AS total_checkins, 
      SUM(checkins.checkins)/COUNT(DISTINCT busniness.business_id) AS average_checkins
FROM `team-8-bs-770-b.Yelp.Yelp_business` AS busniness
  INNER JOIN `team-8-bs-770-b.Yelp.Checkins` AS checkins
  USING ( business_id )
WHERE categories LIKE '%Restaurant%' OR categories LIKE '%restaurant%'
GROUP BY city, state
HAVING AVG(stars) > 3.5 AND total_number_of_restaurants > 813
ORDER BY average_checkins DESC
LIMIT 5


city,total_number_of_restaurants,total_checkins,average_checkins
Las Vegas,5741,3286131,572.396969169
Scottsdale,1398,756653,541.23962804
Phoenix,3508,1097825,312.948973774
Madison,970,127069,130.998969072
Pittsburgh,2141,276951,129.355908454


Creating the table “LasVegas_Nested_Table”

In [ ]:
%%bq query

SELECT
  business_id,
  name,
  SPLIT(categories, ";") AS All_Cat
FROM
  `team-8-bs-770-b.Yelp.Yelp_business`
WHERE
  city = "Las Vegas"
  AND state = "NV"
  AND categories LIKE ('%Restaurants%')

# This query helps us split the “categories” column from the Yelp_business table into separate categories per restaurant

Creating the table “Phoenix_Nested_Cat”

In [ ]:
%%bq query

SELECT
  business_id,
  name,
  SPLIT(categories, ";") AS All_Cat
FROM
  `team-8-bs-770-b.Yelp.Yelp_business`
WHERE
  city = "Phoenix"
  AND state = "AZ"
  AND categories LIKE ('%Restaurants%')

# This query helps us split the “categories” column from the Yelp_business table into separate categories per restaurant

Creating the table “Scottsdale_Nested_Cat”

In [ ]:
%%bq query

SELECT
  business_id,
  name,
  SPLIT(categories, ";") AS All_Cat
FROM
  `team-8-bs-770-b.Yelp.Yelp_business`
WHERE
  city = "Scottsdale"
  AND state = "AZ"
  AND categories LIKE ('%Restaurants%')
 
# This query helps us split the “categories” column from the Yelp_business table into separate categories per restaurant

Creating the table “LasVegas_Cat”

In [ ]:
%%bq query

SELECT business_id, each_cat 
FROM `team-8-bs-770-b.By_City.LasVegas_Nested_Table` 
, UNNEST( All_cat ) each_cat

# This query helps us unnest the array in “LasVegas_Nested_Table”

Creating the table “Phoenix_Cat”

In [ ]:
%%bq query

SELECT business_id, each_cat 
FROM `team-8-bs-770-b.By_City.Phoenix_Nested_Cat` 
, UNNEST( All_cat ) each_cat

# This query helps us unnest the array in “Phoenix_Nested_Cat”

Creating the table “Scottsdale_Cat”

In [ ]:
%%bq query

SELECT business_id, each_cat 
FROM `team-8-bs-770-b.By_City.Scottsdale_Nested_Cat` 
, UNNEST( All_cat ) each_cat

# This query helps us unnest the array in “Scottsdale_Nested_Cat”

Creating table Total_checkins to calculate the sum of check-ins for each business.


In [ ]:
%%bq query
CREATE TABLE IF NOT EXISTS By_City.Total_checkins
AS
SELECT business_id, sum(checkins) as total_checkins 
FROM `team-8-bs-770-b.Yelp.Checkins` group by business_id

## This table contains business_id and their counts from Yelp.Checkins table

Created 3 temporary tables, one per city, joining the columns business_id, name, city, stars, review_count, total_checkins, from tables Yelp_business and total_checkins: Las Vegas, Phoenix and Scottsdale.


In [ ]:
%%bq query

--For Las Vegas

CREATE TABLE IF NOT EXISTS  Saved_queries.Temp_Vegas
AS
SELECT business_id, name, city, stars, review_count, total_checkins
FROM `team-8-bs-770-b.Yelp.Yelp_business` 
  INNER JOIN `team-8-bs-770-b.By_City.Total_checkins` 
  USING (business_id)
WHERE city = 'Las Vegas' AND
    state = 'NV' AND
    (categories LIKE '%Restaurant%' OR categories LIKE '%restaurant%')

In [ ]:
%%bq query

--For Phoenix

CREATE TABLE IF NOT EXISTS  Phoenix_in_process1
AS
SELECT business_id, name, city, stars, review_count, total_checkins
FROM `team-8-bs-770-b.Yelp.Yelp_business` 
  INNER JOIN `team-8-bs-770-b.By_City.Total_checkins` 
  USING (business_id)
WHERE city = 'Phoenix' AND
    state = 'AZ' AND
    (categories LIKE '%Restaurant%' OR categories LIKE '%restaurant%')

In [ ]:
%%bq query

--For Scottsdale

CREATETABLE IF NOT EXISTS  Categories_By_City.Scottsdale_in_process1
AS
SELECT business_id, name, city, stars, review_count, total_checkins
FROM `team-8-bs-770-b.Yelp.Yelp_business` 
  INNER JOIN `team-8-bs-770-b.By_City.Total_checkins` 
  USING (business_id)
WHERE city = 'Scottsdale' AND
    state = 'AZ' AND
    (categories LIKE '%Restaurant%' OR categories LIKE '%restaurant%')

Then created a new table per city, by joining prior table to their respective category and business_ids.

In [ ]:
%%bq query

--For Las Vegas:

CREATE TABLE IF NOT EXISTS  Final_Tables.LasVegas
AS
SELECT * FROM `team-8-bs-770-b.Saved_queries_Temp_Vegas`
INNER JOIN `team-8-bs-770-b.Saved_queries.LasVegas_Cat`
USING (business_id)

In [ ]:
%%bq query

--For Phoenix:

CREATETABLE IF NOT EXISTS  Phoenix_in_process2
AS 
SELECT * FROM `team-8-bs-770-b.Categories_By_City.Phoenix_in_process1`
INNER JOIN `team-8-bs-770-b.Saved_queries.Phoenix_Cat`
USING (business_id)

In [ ]:
%%bq query

--For Scottsdale

CREATETABLE IF NOT EXISTS  Categories_By_City.Scottsdale_in_process2
AS
SELECT * FROM `team-8-bs-770-b.Categories_By_City.Scottsdale_in_process1`
INNER JOIN `team-8-bs-770-b.Saved_queries.Scottsdale_Cat`
USING (business_id)

Cleaning process for 7 attributes being used

In [3]:
%%bq query
SELECT business_id, stars,
CASE WHEN BusinessAcceptsCreditCards = 'TRUE' THEN TRUE
ELSE FALSE END AS BusinessAcceptsCreditCards,
CASE WHEN BusinessParking_garage = 'TRUE' THEN TRUE
ELSE FALSE END AS BusinessParking_garage,
CASE WHEN WheelchairAccessible = 'TRUE' THEN TRUE
ELSE FALSE END AS WheelchairAccessible,
CASE WHEN BikeParking = 'TRUE' THEN TRUE
ELSE FALSE END AS BikeParking,
CASE WHEN Alcohol = 'TRUE' THEN TRUE
ELSE FALSE END AS Alcohol,
CASE WHEN RestaurantsReservations = 'TRUE' THEN TRUE
ELSE FALSE END AS RestaurantsReservations,
CASE WHEN HappyHour = 'TRUE' THEN TRUE
ELSE FALSE END AS HappyHour,
CASE WHEN OutdoorSeating = 'TRUE' THEN TRUE
ELSE FALSE END AS OutdoorSeating
FROM `team-8-bs-770-b.Yelp.RestauranWatt`



business_id,stars,BusinessAcceptsCreditCards,BusinessParking_garage,WheelchairAccessible,BikeParking,Alcohol,RestaurantsReservations,HappyHour,OutdoorSeating
X6lidaXqFC4SGRAxA97yRA,2.0,False,False,False,False,False,False,False,False
Vgy6FfL-l1Ikw7dmJjXr5w,2.0,False,False,False,False,False,False,True,False
8QmYK1h-K_Km42BCNd-hJw,2.0,False,False,False,False,False,False,False,False
M7xYOQztl4Db3Hp6LaAUNQ,2.0,False,False,False,False,False,False,False,True
s-Ma9P-hm4eLBlJy1PnJ3Q,2.0,False,False,True,False,False,False,False,False
feJ_LjOUElLmG3NLNbBFJg,2.0,False,False,False,False,False,False,False,False
OeeQkxeqdYTP5qNKGHznHQ,2.0,False,False,False,False,False,False,False,False
ZTnFd587gd21wuGDDDD3Ng,2.0,False,False,False,False,False,False,False,False
ArRALWe_LLc89RdBeQKscw,2.0,False,False,True,False,False,False,False,False
GeeR1ScER2WpFoJs1sTjsA,2.0,False,False,False,False,False,False,False,False


To observe the most used comments we joined the reviews table to the Yelp_business which contains the basic information for each business in the Yelp data base and filtered the businesses which are considered restaurants.

In [1]:
%%bq query
SELECT r.stars, r.text
FROM `team-8-bs-770-b.Yelp.Yelp_reviews` as r
inner join `team-8-bs-770-b.Yelp.Yelp_business` as b 
using (business_id)
where b.categories LIKE ('%Restaurants%')

stars,text
1,"What a terrible experience and we didn't even get into the restaurant! I stopped to eat at Hooters with 3 friends after a concert last night. We had to wait in line for ab 20 minutes even though there were plenty of tables that appeared empty. A group of young men came by the entrance as we were waiting in line and appeared very drunk, were rowdy and at one point one of the young men pulled his pants down while one of his friends extinguished a cigarette in his buttocks. A few minutes later the same thing happened. The young men then got in line behind us for a few minutes, disappeared from said line and then appeared at the front of the line and were escorted right into the restaurant. Many individuals in the line were upset. An older man was talking to the manager and the manager did nothing to apologize to anyone for the ordeal and he did it with a smirk on his face. What a terrible display of customer service. It's no wonder the casino wasn't that full...if that type of service runs through the whole establishment."
1,This is that worst place to eat ...coffee lid was covered in dust and washroom was not cleaned for Days! Do not eat at here One star is still to much Go across the street for better and cleaner service aka pho Linh
1,"My friend and I came here because of the grand opening BOGO. The interior was nice and comfortable. However, the food and service was not great. We came at 10:30pm when this place closes at midnight. We ordered our first round which came pretty quickly considering the restaurant wasn't full and busy. The sushi was ok but most of the nigiri was not fresh especially the squid. I took a bite and had to spit it because it was mushy and it smelled foul. Second round came and apparently it's already last call at 11:00pm so we were rushed to order everything at once. Our second round most of the orders were incorrect such as my yellowtail hand roll came out as albacore and then suddenly we had edamame when we didn't write it on the paper. The funny thing is when you turn in the paper they sit in front of you and review what you ordered and they still get it wrong. No refills on our tea from 3 different people and when we did get a refill a busser brought us one cup of tea leaving my friends cup empty. When we asked to get a refill the manager came back and said they had shut off their tea machine and asked if water was ok...what other options do we have? They charged us $2 per tea and they think because they're closing up it's ok. They didn't even remove one tea charge from the bill for that. The worst part? My friend and I got pretty sick from eating here afterwards. Let's just say we are both dehydrated as a result of this. I would not come back here even if they had another BOGO."
1,"The food was just meh. We had Pad Thai. However there was no tofu, and we thought that they didn't make it with authentic noodles. Plus the chicken tasted like BBQ chicken and there were very little peanuts. None of the ingredients were really correct. The service was quite slow. I.e. We had to get up to ask for more water. And the server slammed down our plates, without taking the appetizer plates first. To top it off, the table was sticky. :("
1,"We used to go to this Chili's a lot because it's right in our neighborhood. The service and food used to be pretty good. For the last six months or so, the service has been terrible. Last night, we went in for the last time. We sat at a table for 15 minutes and were never even acknowledged by any staff person, so we got up and left. Actually, no one even noticed that we left either! That's pretty bad! This place will get no more of my business!"
1,Way overpriced and everything was salty. My $20 Bloody Mary wasn't worth $5. My husbands $23 tuna tartar was 4 chips with a bite of just ok tuna on each.
1,"Food wasn't that great, cheese had no flavor at all. The consistency was more of water than cheese. They charged me another item and then it took about half an hou

We splitted the reviews by separating each of them where there was a “space” between the words of each review.

In [2]:
%%bq query
SELECT stars, split(text, " ") AS words
FROM `team-8-bs-770-b.Yelp.RestauranReviews`

stars,words
3,"['A', 'very', 'convenient', 'location', 'situated', 'just', 'on', 'the', 'north-east', 'corner', 'of', 'Yonge', '&', 'Bloor.', 'This', 'nice', 'little', 'boutique', 'restaurant', 'is', 'quite', 'visible', 'thanks', 'to', 'a', 'standing', 'sign', 'that', 'says', '""Ichiriki', 'sushi', 'restaurant"".\n\nThe', 'service', 'in', 'this', 'restaurant', 'is', 'quick.', 'I', 'ordered', 'a', 'salmon', 'and', 'sashimi', 'dish,', 'with', 'steam', 'rice', 'for', 'an', 'extra', 'dollar.', 'The', 'portions', 'are', 'surprisingly', 'huge.', 'On', 'my', 'dish,', 'there', 'are', 'three', 'pieces', 'of', 'sashimi,', 'three', 'pieces', 'of', 'sushi', 'roll,', 'garden', 'salad', 'on', 'the', 'side,', 'two', 'large', 'pieces', 'of', 'salmon', 'and', 'a', 'small', 'dish', 'of', 'tuna', 'salad.', 'The', 'tuna', 'salad', 'mini-dish', 'was', 'refreshing', 'and', 'tasty.', 'The', 'sushi', 'has', 'wasabi', 'inside,', 'which', 'is', 'prepared', 'the', 'correct', 'authentic', 'way.', '\n\nThe', 'setting', 'of', 'the', 'restaurant', 'is', 'suitable', 'for', 'a', 'jazz', 'band,', 'as', 'there', 'are', 'many', 'jazz', 'photos', 'posted', 'on', 'the', 'wall.', 'I', 'was', 'told', 'by', 'the', 'waitress', 'that', 'jazz', 'nights', 'take', 'place', 'in', 'the', 'restaurant', 'on', 'the', 'weekends.', 'I', 'would', 'raise', 'an', 'extra', 'star', 'rating', 'if', 'I', 'come', 'here', 'one', 'night', 'for', 'their', 'jazz', 'performance.']"
5,"['Great', 'breakfast', 'spot.', 'Has', 'free', 'parking;', 'we', 'got', 'here', 'at', '8:45', 'am', 'on', 'a', 'Saturday,', 'beautiful', 'Saturday,', 'which', 'is', 'an', 'important', 'detail', 'when', 'venturing', 'to', 'the', 'Strip', 'District,', 'we', 'had', 'four', 'in', 'our', 'group', 'and', 'sat', 'immediately.', 'Service', 'was', 'friendly', 'and', 'fast.', 'Atmosphere', 'is', 'loud,', 'which', 'I', 'personally', 'really', 'like,', 'and', 'it', 'was', 'really', 'clean.', 'Chocolate', 'chip', 'pancakes', 'are', 'on', 'the', 'menu,', 'a', 'plus', 'if', 'you', 'have', 'lucky', 'kids.', 'I', 'do.', 'Coffee', 'was', 'smooth', 'and', 'the', 'bread', 'used', 'for', 'my', 'breakfast', 'sandwich', 'was', 'so', 'good', 'and', 'fresh.', 'Booth,', 'table,', '&', 'bar', 'seating.', ""I've"", 'been', 'looking', 'for', 'a', 'really', 'great', 'breakfast', 'place', 'to', 'visit', 'on', 'the', 'regular', 'and', 'Kelly', ""O's"", 'will', 'be', 'at', 'the', 'top', 'of', 'my', 'list.', 'There', 'is', 'really', 'nothing', 'negative', 'to', 'say', 'about', 'our', 'experience.']"
5,"[""I've"", 'gone', 'here', '5', 'times', 'and', 'loved', 'this', 'place', 'each', 'time', ""I've"", 'gone.', 'They', 'have', 'really', 'good', 'smoothies', 'and', 'their', 'sandwiches', 'are', 'huge.', 'And', 'I', 'think', ""they're"", 'starting', 'to', 'sell', 'sushi', 'now', 'so', ""that's"", 'a', 'plus']"
5,"[""Mastro's"", 'was', 'recommended', 'by', 'a', 'co-worker', 'who', 'went', 'to', 'one', 'of', 'the', 'locations', 'in', 'CA', 'and', 'absolutely', 'loved', 'it.', 'I', 'made', 'reservations', 'for', 'a', 'group', 'of', 'friends', 'and', 'I,', 'and', 'despite', 'some', 'initial', 'resistance', 'from', 'my', 'fellow', 'Yelper', 'Lauren', 'M.,', 'we', 'all', 'had', 'a', 'fabulous', 'time', 'and', 'were', 'more', 'than', 'happy', 'with', 'our', 'decision', 'to', 'come', 'here!\n\nOur', 'server,', 'Lionel,', 'was', 'amazing', 'the', 'whole', 'evening.', 'He', 'was', 'friendly,', 'helpful,', 'and', 'attentive.', 'He', 'recommended', 'we', 'get', 'a', 'customized', 'seafood', 'tower,', 'and', 'everyone', 'in', 'the', 'group', 'was', 'very', 'impressed.\n\nWe', 'were', 'also', 'impressed', 'with', 'our', 'meals.', 'I', 'got', 'the', 'bone-in', 'filet,', 'and', 'two', 'of', 'my', 'friends', 'split', 'the', 'tomahawk', 'special.', 'My', 'other', 'friends', 'got', 'fish.', 'My', 'filet', 'melted', 'in', 'my', 'mouth', 'and', 'was', 'cooked', 'perfectly.', 'I', ""don't"", 'typically', 'get', 'filet', 'when', 'I', 'go', 'to', 'a', 'steakhouse,', 'but'